In [1]:
pip list

Package                   Version
------------------------- --------------
anyio                     4.8.0
appnope                   0.1.4
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.4
attrs                     25.1.0
babel                     2.17.0
beautifulsoup4            4.13.3
bleach                    6.2.0
certifi                   2025.1.31
cffi                      1.17.1
charset-normalizer        3.4.1
comm                      0.2.2
contourpy                 1.3.1
cycler                    0.12.1
debugpy                   1.8.12
decorator                 5.1.1
defusedxml                0.7.1
executing                 2.2.0
fastjsonschema            2.21.1
fonttools                 4.56.0
fqdn                      1.5.1
h11                       0.14.0
httpcore                  1.0.7
httpx                     0.28.1
idna                      3.10
ipykernel     

In [2]:
pip show statsmodels

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install statsmodels

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 18.9 MB/s eta 0:00:0031m9.2 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [7]:
pip show statsmodels

Name: statsmodels
Version: 0.14.4
Summary: Statistical computations and models for Python
Home-page: https://www.statsmodels.org/
Author: 
Author-email: 
License: BSD License
Location: /Users/chuqing/Desktop/LazyProgrammer/machine_learning_exercises/ab_testing/myenv/lib/python3.13/site-packages
Requires: numpy, packaging, pandas, patsy, scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [8]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
from scipy.stats import norm
from statsmodels.stats.weightstats import ztest

In [10]:
help(ztest)

Help on function ztest in module statsmodels.stats.weightstats:

ztest(x1, x2=None, value=0, alternative='two-sided', usevar='pooled', ddof=1.0)
    test for mean based on normal distribution, one or two samples

    In the case of two samples, the samples are assumed to be independent.

    Parameters
    ----------
    x1 : array_like, 1-D or 2-D
        first of the two independent samples
    x2 : array_like, 1-D or 2-D
        second of the two independent samples
    value : float
        In the one sample case, value is the mean of x1 under the Null
        hypothesis.
        In the two sample case, value is the difference between mean of x1 and
        mean of x2 under the Null hypothesis. The test statistic is
        `x1_mean - x2_mean - value`.
    alternative : str
        The alternative hypothesis, H1, has to be one of the following

           'two-sided': H1: difference in means not equal to value (default)
           'larger' :   H1: difference in means larger than va

In [2]:
np.random.seed(0)

In [3]:
# one-sample test
N = 100
mu = 0.2
sigma = 1
x = np.random.randn(N) * sigma + mu

In [4]:
# two-sided z-test
ztest(x) # default compare to 0

(np.float64(2.5648404153513686), np.float64(0.010322326848815837))

In [5]:
# two-sided z-test
mu_hat = x.mean()
sigma_hat = x.std(ddof=1)
z = mu_hat / (sigma_hat / np.sqrt(N))
# p_right = 1 - norm.cdf(np.abs(z)) #cdf not ppf 
p_right = norm.sf(np.abs(z)) #cdf not ppf 
p_left = norm.cdf(-np.abs(z)) # use np.abs not the default abs for abs elementwisely
p = p_left + p_right
z, p

(np.float64(2.564840415351368), np.float64(0.010322326848815847))

In [6]:
# one-sided z-test
ztest(x, alternative='larger')

(np.float64(2.5648404153513686), np.float64(0.005161163424407918))

In [7]:
# one-sided z-test
mu_hat = x.mean()
sigma_hat = x.std(ddof=1)
z = mu_hat / (sigma_hat / np.sqrt(N))
p = norm.sf(z) #no need for abs
z, p

(np.float64(2.564840415351368), np.float64(0.005161163424407924))

In [8]:
# null under a defferent reference value
mu0 = 0.2
ztest(x, value=mu0)

(np.float64(0.5904283402851699), np.float64(0.5549035151647227))

In [9]:
# null under a defferent reference value
mu_hat = x.mean()
sigma_hat = x.std(ddof=1)
z = (mu_hat - mu0) / (sigma_hat / np.sqrt(N))
# p_right = 1 - norm.cdf(np.abs(z)) #cdf not ppf 
p_right = norm.sf(np.abs(z)) #cdf not ppf 
p_left = norm.cdf(-np.abs(z)) # use np.abs not the default abs for abs elementwisely
p = p_left + p_right
z, p

(np.float64(0.5904283402851698), np.float64(0.5549035151647228))

In [10]:
#check significane threshold
def experiment(): #check false positive
    mu0 = 0.2
    x = norm.rvs(loc=mu, scale=sigma, size=N)
    _, p = ztest(x, value=mu0)
    return p<0.05

def multi_experiments(M): #check false positive rate
    results = [experiment() for _ in range(M)] 
    return np.mean(results) 

multi_experiments(10000)

np.float64(0.0505)

In [11]:
# two-sample test
N0 = 100
mu0 = 0.2
sigma0 = 1
x0 = np.random.randn(N0) * sigma0 + mu0

N1 = 100
mu1 = 0.5
sigma1 = 1
x1 = np.random.randn(N1) * sigma1 + mu1

In [12]:
#two-sided test
ztest(x0, x1)

(np.float64(-4.203806302978261), np.float64(2.6246356365038274e-05))

In [13]:
#two-sided test
mu_hat0 = x0.mean()
mu_hat1 = x1.mean()
dmu_hat = mu_hat0 - mu_hat1
s2_hat0 = x0.var(ddof=1)
s2_hat1 = x1.var(ddof=1)
s_hat = np.sqrt(s2_hat0 / N0 + s2_hat1 / N1) #s_hat not s2_hat
z = dmu_hat / s_hat
p_left = norm.cdf(-np.abs(z))
p_right = norm.sf(np.abs(z))
p = p_left + p_right
z, p

(np.float64(-4.203806302978261), np.float64(2.6246356365038274e-05))

In [14]:
# one-sided test
ztest(x0, x1, alternative = 'smaller')

(np.float64(-4.203806302978261), np.float64(1.3123178182519137e-05))

In [19]:
# show false positive rate at 5% significance level
num_tests = 10000
results = np.zeros(num_tests)
for i in range(num_tests): 
    x1 = np.random.randn(100)
    x2 = np.random.randn(100)
    z, p = ztest(x1, x2) #H_0 is true
    results[i] = (p<0.05) #false positive
    # not need to end for
print(results.mean())

0.0539


In [15]:
# show that false positive occurs 5% of the time
num_tests = 10000
results = np.zeros(num_tests)
for i in range(num_tests): 
    x1 = np.random.randn(100)
    x2 = np.random.randn(100)
    z, p = ztest(x1, x2, alternative='smaller') #H_0 is true
    results[i] = (p<0.05) #false positive
print(results.mean())

0.0524


In [67]:
norm.ppf(0.95)

np.float64(1.644853626951472)

In [20]:
# show false positive rate at 5% significance level
num_tests = 10000
results = np.zeros(num_tests)
for i in range(num_tests): 
    x1 = np.random.randn(100)
    x2 = np.random.randn(100)
    z, p = ztest(x1, x2, alternative='smaller') #H_0 is true
    results[i] = (p<0.05) #false positive
    # not need to end for
print(results.mean())

0.0519


In [21]:
# show false positive rate at 5% significance level
num_tests = 10000
results = np.zeros(num_tests)
for i in range(num_tests): 
    x1 = np.random.randn(100)+0.1
    x2 = np.random.randn(100)
    z, p = ztest(x1, x2, alternative='smaller') #H_0 is true
    results[i] = (p<0.05) #false positive
    # not need to end for
print(results.mean())

0.0093


In [22]:
# show false negative rate at 5% significance level
num_tests = 10000
results = np.zeros(num_tests)
for i in range(num_tests): 
    x1 = np.random.randn(100)
    x2 = np.random.randn(100)+0.1
    z, p = ztest(x1, x2, alternative='smaller') #H_1 is true
    results[i] = not (p<0.05) #false negative
    # not need to end for
print(results.mean())

0.8278
